In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [ ]:
def add_location(frame, location):
    frame["location"] = location
    return frame

In [ ]:
def removeNaN(frame):
    frame = frame.fillna(method="ffill")
    frame = frame.fillna(method="bfill")
    frame = frame.fillna(value=0)
    return frame

In [ ]:
def averageFeatures(frame):
    frame["date_forecast"] = frame["date_forecast"].apply(lambda a : a.replace(minute=0))
    return frame.groupby(["date_forecast"])[frame.iloc[:,1:].columns].mean().reset_index()

Load and clean data

In [ ]:
train_a = pd.read_parquet('A/train_targets.parquet')
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')

train_b = pd.read_parquet('B/train_targets.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')

train_c = pd.read_parquet('C/train_targets.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')

train_a = add_location(train_a, "A")
train_b = add_location(train_b, "B")
train_c = add_location(train_c, "C")
X_train_estimated_a = X_train_estimated_a.iloc[:,1:]
X_train_estimated_b = X_train_estimated_b.iloc[:,1:]
X_train_estimated_c = X_train_estimated_c.iloc[:,1:]
X_train_estimated_a = add_location(averageFeatures(removeNaN(X_train_estimated_a)), "A")
X_train_estimated_b = add_location(averageFeatures(removeNaN(X_train_estimated_b)), "B")
X_train_estimated_c = add_location(averageFeatures(removeNaN(X_train_estimated_c)), "C")
X_train_observed_a = add_location(averageFeatures(removeNaN(X_train_observed_a)), "A")
X_train_observed_b = add_location(averageFeatures(removeNaN(X_train_observed_b)), "B")
X_train_observed_c = add_location(averageFeatures(removeNaN(X_train_observed_c)), "C")

In [ ]:
#Beklager ekstremt nasty kode her


targets = pd.concat([train_a, train_b, train_c]).sample(frac=1)
features = pd.concat([
                     X_train_observed_a, 
                     X_train_observed_b, 
                     X_train_observed_c,
                     X_train_estimated_a,
                    X_train_estimated_b,
                    X_train_estimated_c]).sample(frac=1)

train_features = features.iloc[:90000,:]
train_features = train_features.rename(columns={"date_forecast" : "time"})
trainset = pd.merge(train_features, targets, how="right", on=["time","location"]).dropna()

eval_features = features.iloc[90000:,:]
eval_features = eval_features.rename(columns={"date_forecast" : "time"})

evalset = pd.merge(eval_features, targets, how="right", on=["time","location"]).dropna()

trainsetX = trainset.iloc[:, :-1].reset_index().iloc[:,1:]
trainsetX["day"] = trainsetX["time"].dt.day_of_year
trainsetX["hour"] = trainsetX["time"].dt.hour


sample567 = trainsetX["time"].apply(lambda time : time.month in[5, 6, 7])
#display(sample_weights)



trainsetX = trainsetX.iloc[:,1:]
trainsetX["location"] = trainsetX["location"].map({"A": 0, "B": 1, "C": 2})


dataMean = trainsetX.mean()
dataStd = trainsetX.std()
trainsetX = ((trainsetX-dataMean)/dataStd).fillna(value=0)
trainsetY = trainset.iloc[:, -1:].reset_index().iloc[:,1:]


evalsetX = evalset.iloc[:, :-1].reset_index().iloc[:, 1:]

evalsetX["day"] = evalsetX["time"].dt.day_of_year
evalsetX["hour"] = evalsetX["time"].dt.hour

#get indexes of the samples from may, june, july
eval567 = evalsetX["time"].apply(lambda time : time.month in[5, 6, 7])

evalsetX = evalsetX.iloc[:,1:]
evalsetX["location"] = evalsetX["location"].map({"A": 0, "B": 1, "C": 2})
evalsetX = ((evalsetX-dataMean)/dataStd).fillna(value=0)
evalsetY = evalset.iloc[:, -1:].reset_index().iloc[:, 1:]



#Visual of features
display(trainsetX)




In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.GaussianNoise(stddev=0.03, seed=42),
  tf.keras.layers.Dense(40, activation="tanh"),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(15, activation="relu"),
  tf.keras.layers.Dense(5, activation="relu")
])


In [ ]:
#weighting for samples from months matching the testset
sample_weights = sample567.map({True : 1.35, False : 1})

model.compile(
  optimizer=tf.keras.optimizers.experimental.Adadelta(learning_rate=0.5),
  loss='mean_absolute_error'
  )

history = model.fit(trainsetX, 
                    trainsetY, 
                    batch_size=500, 
                    epochs=300,
                    validation_data=(evalsetX, evalsetY),
                    sample_weight=sample_weights
                    )

In [ ]:

evalset567 = evalsetX.join(evalsetY)
evalset567 = evalset567[eval567]
evalset567X = evalset567.iloc[:,:-1]
evalset567Y = evalset567.iloc[:,-1:]

#results only in may, june, july(since the test set only contains data from those months)
#this should closely match the expected score on kaggle
results1 = model.evaluate(evalset567X, evalset567Y)

#results for entire eval set
results2 = model.evaluate(evalsetX, evalsetY)

In [ ]:
#lagre 

preds = model.predict(evalsetX)
compare = pd.concat([evalsetY, pd.DataFrame(preds)], axis=1)
compare.to_csv("test1.csv")

In [ ]:
#Igjen beklager for nasty kode

X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('A/X_test_estimated.parquet')

X_test_estimated_a = X_test_estimated_a.iloc[:,1:]
X_test_estimated_b = X_test_estimated_b.iloc[:,1:]
X_test_estimated_c = X_test_estimated_c.iloc[:,1:]
X_test_estimated_a = add_location(averageFeatures(removeNaN(X_test_estimated_a)), "A")
X_test_estimated_b = add_location(averageFeatures(removeNaN(X_test_estimated_b)), "B")
X_test_estimated_c = add_location(averageFeatures(removeNaN(X_test_estimated_c)), "C")


X_test_estimated = pd.concat([X_test_estimated_a, X_test_estimated_b,X_test_estimated_c]).rename(columns={"date_forecast" : "time"})


X_test_estimated_a["location"] = "A"
X_test_estimated_b["location"] = "B"
X_test_estimated_c["location"] = "C"

parse_dates = ['time']
X_test_targets = pd.read_csv("test.csv", parse_dates=parse_dates)

X_test = pd.merge(X_test_estimated, X_test_targets, on=["time", "location"], how="right")

X_test = X_test.iloc[:,0:-2]
X_test["day"] = X_test["time"].dt.day_of_year
X_test["hour"] = X_test["time"].dt.hour
X_test = X_test.iloc[:,1:]
X_test["location"] = X_test["location"].map({"A": 0, "B": 1, "C": 2})
X_test = ((X_test-dataMean)/dataStd).fillna(value=0)

X_test["snow_density:kgm3"] = 0

compare = pd.concat([X_test, evalsetX])


In [ ]:
display(X_test.describe()-evalsetX.describe())

In [ ]:

test_preds = pd.read_csv("sample_submission.csv")
test_preds["prediction"] = model.predict(X_test)
test_preds.to_csv("preds/NN_locationTimeInput_Ada_correctNormalization_sampleWeightedByMonth.csv", index=False)